# Bulk processing

Take all the jump files in a data lake and process them as a group.  Produce the mean results for scores, max speed, curves, and the 5-second partial results.

Conforming to Lucyfer's default configuration, the data lake starts at the `./data` directory and includes any and all files below it.

---
## Prepare the environment

In [ ]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.fs1 import aggregateResults
from ssscoring.fs1 import convertFlySight2SSScoring
from ssscoring.fs1 import dropNonSkydiveDataFrom
from ssscoring.fs1 import getAllSpeedJumpFilesFrom
from ssscoring.fs1 import getSpeedSkydiveFrom
from ssscoring.fs1 import isValidJump
from ssscoring.fs1 import jumpAnalysisTable
from ssscoring.fs1 import processAllJumpFiles
from ssscoring.fs1 import totalResultsFrom
from ssscoring.fs1 import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import ipywidgets as widgets
import pandas as pd

In [ ]:
DATA_LAKE_ROOT = './data'

## Get a list of all FlySight files in the data lake

This also discards all files that don't reflect a valid jump:

- Detect the files by size and discard any file smaller than `MIN_JUMP_FILE_SIZE`
- Detect that each file has the FlySight header on the first line

In [ ]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

---
## Process all files

In [ ]:
jumpResults = processAllJumpFiles(jumpFiles)

---
## Results

In [ ]:
aggregate = aggregateResults(jumpResults)
sum = totalResultsFrom(aggregate)
display(aggregate)
display(sum)

In [ ]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in jumpResults.keys():
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        graphJumpResult(initializePlot(resultRef),
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

## All jumps

In [ ]:
bp.show(allJumpsPlot)